In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.metrics import auc
import cv2

def deletion_curve(image, segments, mask_order, predict_fn, label, steps=20, mode='zero'):
    """
    Xoá dần các vùng quan trọng và đo xác suất dự đoán.
    mode: 'zero' hoặc 'mean'
    """
    perturbed = image.copy()
    scores = []
    mean_value = np.mean(image, axis=(0, 1), keepdims=True).astype(np.uint8)

    for step in range(1, steps + 1):
        num_regions = int(len(mask_order) * step / steps)
        current_mask = mask_order[:num_regions]

        for region in current_mask:
            if mode == 'zero':
                perturbed[segments == region] = 0
            else:
                perturbed[segments == region] = mean_value

        probs = predict_fn([perturbed])
        score = probs[0][label]
        scores.append(score)

    print(f"Scores: {scores}")
    x = np.linspace(0, 1, len(scores))
    auc_score = auc(x, scores)

    return auc_score



In [ ]:
def min_subset(image, segments, weights, predict_fn, label, threshold=0.9, mode='zero'):
    """
    Tìm số lượng vùng nhỏ nhất giữ được kết quả dự đoán đủ tốt.
    """
    filtered_regions = [(i, w) for i, w in weights.items() if w > 0.01]
    sorted_regions = sorted(filtered_regions, key=lambda x: -x[1])
    original_pred = predict_fn([image])[0][label]  # Xác suất dự đoán ban đầu

    # Tính giá trị trung bình nếu dùng mode 'mean'
    mean_value = np.mean(image, axis=(0, 1), keepdims=True).astype(np.uint8)  # (1, 1, 3)

    for k in range(1, len(sorted_regions) + 1):
        # Tạo ảnh bị che ban đầu: toàn bộ bằng 0 hoặc bằng mean
        if mode == 'zero':
            temp = np.zeros_like(image)
        else:
            temp = np.ones_like(image) * mean_value  # mean_value sẽ được broadcast

        # Lấy k vùng quan trọng nhất
        selected = [r[0] for r in sorted_regions[:k]]

        # Khôi phục lại các vùng đã chọn từ ảnh gốc
        for region in selected:
            temp[segments == region] = image[segments == region]

        # Dự đoán và so sánh xác suất
        prob = predict_fn([temp])[0][label]
        if prob >= threshold * original_pred:
            return k  # Đạt ngưỡng

    return len(sorted_regions)  # Nếu không vùng nào đạt ngưỡng thì trả về toàn bộ


In [ ]:
def pointing_game(importance_map, gt_mask):
    max_pos = np.unravel_index(np.argmax(importance_map), importance_map.shape)
    return int(gt_mask[max_pos] > 0)  # 1 nếu đúng, 0 nếu sai

def extended_pointing_game(importance_map, gt_mask, top_percent=0.01):
    num_pixels = int(np.prod(importance_map.shape) * top_percent)
    flat = importance_map.flatten()
    top_indices = np.argpartition(flat, -num_pixels)[-num_pixels:]
    top_coords = np.unravel_index(top_indices, importance_map.shape)
    hits = gt_mask[top_coords] > 0
    return np.sum(hits) / num_pixels

In [1]:
import os

def load_yolo_mask(label_path, image_shape, class_filter=None):
    h, w = image_shape[:2]
    mask = np.zeros((h, w), dtype=np.uint8)
    if not os.path.exists(label_path):
        return mask  # No annotation = all background

    with open(label_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            parts = line.strip().split()
            if len(parts) != 5:
                continue
            class_id, x_center, y_center, bbox_width, bbox_height = map(float, parts)
            if class_filter is not None and int(class_id) != class_filter:
                continue  # bỏ qua lớp không mong muốn

            x_center *= w
            y_center *= h
            bbox_width *= w
            bbox_height *= h

            x1 = int(x_center - bbox_width / 2)
            y1 = int(y_center - bbox_height / 2)
            x2 = int(x_center + bbox_width / 2)
            y2 = int(y_center + bbox_height / 2)

            x1 = max(0, x1)
            y1 = max(0, y1)
            x2 = min(w - 1, x2)
            y2 = min(h - 1, y2)

            mask[y1:y2, x1:x2] = 1
    return mask


In [ ]:
import pandas as pd
import numpy as np

# Load CSV
df = pd.read_csv("D:\BTXRD-Code\Object-Detection\LIME_runs\yolo_lime_result.csv")

# Replace values < 0 with NaN
numeric_cols = df.select_dtypes(include=[np.number]).columns
df[numeric_cols] = df[numeric_cols].where(df[numeric_cols] > 0, np.nan)

# Group by 'label' and compute mean
result = df.groupby('label')[numeric_cols].mean()

print(result)